In [1]:
!pip install fastapi uvicorn nest-asyncio python-multipart timm opencv-python

# MiDaS & model download
!pip install git+https://github.com/isl-org/MiDaS.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
!git clone https://github.com/isl-org/MiDaS.git



Cloning into 'MiDaS'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 622 (delta 183), reused 136 (delta 136), pack-reused 377 (from 2)
Receiving objects: 100% (622/622), 3.44 MiB | 3.62 MiB/s, done.
Resolving deltas: 100% (246/246), done.


In [3]:
import sys
sys.path.append('/content/MiDaS')


In [4]:
!wget https://huggingface.co/Intel/dpt-large/resolve/main/dpt_large-midas-2f21e586.pt -O /content/dpt_large-midas.pt



--2025-08-02 10:50:32--  https://huggingface.co/Intel/dpt-large/resolve/main/dpt_large-midas-2f21e586.pt
Resolving huggingface.co (huggingface.co)... 13.35.202.97, 13.35.202.121, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.97|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-08-02 10:50:32 ERROR 404: Not Found.



In [5]:
import torch
import cv2
import numpy as np
from torchvision.transforms import Compose
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import Response

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MiDaS model
model_type = "MiDaS_small"  # options: "DPT_Large", "DPT_Hybrid", "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.to(device)
midas.eval()

# Load transforms
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.l

In [6]:
app = FastAPI()

@app.post("/predict")
async def predict_depth(image: UploadFile = File(...)):
    contents = await image.read()
    img_np = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)

    # Preprocess
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_tensor = transform(img_rgb).to(device)

    # Predict
    with torch.no_grad():
        prediction = midas(input_tensor)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False
        ).squeeze()

    # Postprocess
    depth_map = prediction.cpu().numpy()
    depth_map = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map = depth_map.astype(np.uint8)
    depth_color = cv2.applyColorMap(depth_map, cv2.COLORMAP_MAGMA)

    _, encoded_img = cv2.imencode('.jpg', depth_color)
    return Response(content=encoded_img.tobytes(), media_type="image/jpeg")

# Allow FastAPI to run in Colab
nest_asyncio.apply()


In [7]:
!pip install pyngrok

In [24]:
!ngrok config add-authtoken 30jCIkrAAzsPmkeTidvl6doSoz7_2e7cW7vaT4zKTWCBzK2d5


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [36]:

from pyngrok import ngrok

public_url = ngrok.connect(7860)
print("🔗 Public URL:", public_url)


🔗 Public URL: NgrokTunnel: "https://a40a9362e082.ngrok-free.app" -> "http://localhost:7860"


In [37]:
uvicorn.run(app, host="0.0.0.0", port=7860)


INFO:     Started server process [693]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7860 (Press CTRL+C to quit)


INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-642' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    

INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2405:201:c41f:e027:2713:ee1b:dfaa:1e70:0 - "POST /predi

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [693]


In [38]:
!pkill -f ngrok

